In [ ]:
from synkit.IO import load_database, rsmi_to_its, rsmarts_to_rsmi
data = load_database('./benchmark.json.gz')
data[0]

In [ ]:
from rdchiral.template_extractor import extract_from_reaction
for value in data[:]:
    r, p = value['smart'].split('>>')
    value['reactants'] = r
    value['products'] = p
    value['_id'] = value['R-id']
    value['rdchiral'] = extract_from_reaction(value)['reaction_smarts']
    value['rc'] = rsmi_to_its(value['smart'], core=True)

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df['rdchiral'].value_counts()

In [ ]:
from synkit.Graph.Feature.wl_hash import WLHash
from synkit.Graph.Matcher.graph_cluster import GraphCluster

for value in data:
    value['wl'] = WLHash().weisfeiler_lehman_graph_hash(value['rc'])
cls = GraphCluster()
result = cls.fit(data, rule_key='rc', attribute_key='wl')

In [ ]:
pd.DataFrame(result)['class'].value_counts()

In [ ]:
from synkit.IO import load_database, rsmi_to_its, rsmi_to_rsmarts
from synkit.Chem.Reaction.canon_rsmi import CanonRSMI
canon = CanonRSMI()
for value in result:
    rsmi = rsmarts_to_rsmi(value['rdchiral'])
    # print(rsmi)
    try:
        value['rsmi_rdchiral'] = canon.canonicalise(rsmi).canonical_rsmi
    except:
        continue
        # value['rsmi_rdchiral']

In [ ]:
pd.DataFrame(result)['rsmi_rdchiral'].value_counts()